In [2]:
import findspark
findspark.init()

In [3]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("MostPopularSuperhero2")
sc = SparkContext(conf = conf)

In [4]:
def extract_friends(super_hero_row):
    super_hero=super_hero_row.split()
    super_hero_id=int(super_hero[0])
    super_hero_friends=[int(sh_id) for sh_id in super_hero[1:]]
    return super_hero_id,super_hero_friends

superhero_friends_raw=sc.textFile("/home/ggomarr/Documents/Education/Udemy_Spark/23_Marvel-Graph.txt")
superhero_friends_count_sorted=(superhero_friends_raw.map(extract_friends)
                                                     .map(lambda x: (x[0],len(x[1])))
                                                     .reduceByKey(lambda x, y: x + y)
                                                     .map(lambda x: (x[1],x[0]))
                                                     .sortByKey(False))

In [5]:
for superhero in superhero_friends_count_sorted.take(10):
    print('{:4d} - {:4d}'.format(superhero[0],superhero[1]))

1933 -  859
1741 - 5306
1528 - 2664
1426 - 5716
1394 - 6306
1386 - 3805
1371 - 2557
1345 - 4898
1289 - 5736
1280 -  403


In [6]:
def prepare_superhero_dict(superhero_names_file):
    import re
    superhero_names_dict={}
    with open(superhero_names_file) as f:
        for superhero_row in f:
            superhero=superhero_row.split(' ',1)
            superhero_id=int(superhero[0])
            superhero_name=re.sub('^[^a-zA-Z0-9\s]*|[^a-zA-Z0-9\s]*$','',superhero[1].strip())
            superhero_names_dict[superhero_id]=superhero_name
    return superhero_names_dict

superhero_names_file="/home/ggomarr/Documents/Education/Udemy_Spark/23_Marvel-Names.txt"
superhero_names_dict=prepare_superhero_dict(superhero_names_file)

In [7]:
for superhero_id in range(1,11):
    print('{:4d} - {}'.format(superhero_id,superhero_names_dict[superhero_id]))

   1 - 24-HOUR MAN/EMMANUEL
   2 - 3-D MAN/CHARLES CHAN
   3 - 4-D MAN/MERCURIO
   4 - 8-BALL
   5 - A
   6 - A'YIN
   7 - ABBOTT, JACK
   8 - ABCISSA
   9 - ABEL
  10 - ABOMINATION/EMIL BLO


In [8]:
for superhero in superhero_friends_count_sorted.take(10):
    print('{:3d} - {:4d} - {}'.format(superhero[0],superhero[1],superhero_names_dict[superhero[1]]))

1933 -  859 - CAPTAIN AMERICA
1741 - 5306 - SPIDER-MAN/PETER PAR
1528 - 2664 - IRON MAN/TONY STARK 
1426 - 5716 - THING/BENJAMIN J. GR
1394 - 6306 - WOLVERINE/LOGAN 
1386 - 3805 - MR. FANTASTIC/REED R
1371 - 2557 - HUMAN TORCH/JOHNNY S
1345 - 4898 - SCARLET WITCH/WANDA 
1289 - 5736 - THOR/DR. DONALD BLAK
1280 -  403 - BEAST/HENRY &HANK& P
